# Change Detection Task: Behavioral data analysis
* Author: Daniel Labbé
* Date: 2018-05-15
* Version: 1.0

## Settings

In [258]:
'''Settings'''
#  Directory of folder named 'data'. Defaults to the script folder if unspecified (e.g. script_location\data).
current_dir = 'D:\\analysis\\pomo\\_PythonPanda' #  '' #  Toggle: Default
items = [2, 3, 4, 5]

#  Console width
CSL = 120

## Prepare Data for Import


In [259]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import fnmatch
import inspect
import code
import builtins as __builtin__
import ipywidgets as widgets

from collections import namedtuple
from sys import version
from os import getcwd, path, walk
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

def print(*args, **kwargs):
    '''We are doing a lot of these'''
    end = '\n' + CSL * '─' + '\n'
    return __builtin__.print(*args, **kwargs, sep='\n', end=end)

#  Version number >2 affects float division. See also PYTHON command line option -Q <arg>
assert( int(version[0][0]) > 2 or '__future__' in sys.modules )

current_dir = current_dir is '' and getcwd() or current_dir  
current_dir = current_dir + path.sep + 'data' + path.sep
print(version, current_dir + '\n')

f = [] # Stackoverflow Recipe
for (dirpath, dirname, filename) in walk(current_dir):
    f.extend(filename)
    break


files = pd.Series(f)
printmd('### Files')
print(files)
printmd('### Subjects')
print(*[file[0:6] for file in files])


3.6.3 |Anaconda, Inc.| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
D:\analysis\pomo\_PythonPanda\data\

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


### Files

0              aledie_py_2016-05-24 16-05-34.717000.txt
1              aleess_py_2016-06-05 11-38-42.321000.txt
2              celbah_py_2018-03-14 17-42-33.247000.txt
3              clasze_py_2016-06-09 10-44-01.087000.txt
4              cribal_py_2016-11-15 17-07-00.819000.txt
5              fradam_py_2016-05-31 17-10-14.130000.txt
6              isahar_py_2018-02-23 15-27-12.294000.txt
7              jenapp_py_2016-08-29 10-07-01.643000.txt
8              jenbom_py_2016-06-02 11-37-25.493000.txt
9              jonhum_py_2016-10-07 12-16-29.988000.txt
10             katwet_py_2016-10-12 15-14-00.294000.txt
11             lenplu_py_2018-02-20 10-58-35.206000.txt
12             lucwil_py_2016-10-19 12-07-22.257000.txt
13             manrei_py_2018-04-18 14-07-55.465000.txt
14             marlie_py_2018-02-21 11-30-03.431000.txt
15    maumcm_JOINED_TWO_FILES_SecondFileReplacedTria...
16             tamkla_py_2016-09-01 14-17-33.982000.txt
17                yjk_py_2016-09-02 10-48-40.036

### Subjects

aledie
aleess
celbah
clasze
cribal
fradam
isahar
jenapp
jenbom
jonhum
katwet
lenplu
lucwil
manrei
marlie
maumcm
tamkla
yjk_py
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [260]:
iter_files = iter(f) #np.nditer(f)

##### DEVELOPMENT NOTES
# New reclass data_* vars to namedtuple
# Used Pandas DataFrame instead of np.genfromtxt

results = namedtuple('Results', ['trials', 'valid', 'missing' 'hit', 'miss', 'fa', 'cr', 'cowans_k'])

for filename in iter_files:
	print('Current file is {}...'.format(filename))
	filename = path.join(current_dir, filename)
	data = pd.read_csv( #np.genfromtxt(
		filename,
		dtype = None,
		names = True,
		delimiter = ',',
		na_values = (999999),
		usecols = [0, 1]#[1, 2, 4, 5, 6, 7, 8, 9, 10]#(1,2,4,5,6,7,8,9,10) # np.genfromtxt expects a tuple here instead .# TODO Ignoring stimulus data for now
	)
	print(data)
	d = data.ndim
	data = np.reshape(data,(data.size, 1))
    
	print('Dim reshaped from {} to {}'.format(d, data.ndim))

	# Print the first 4 rows
	#print('File Preview: ')
	#print(data[0:4])

	# Remove illegal responses
	l = len(data)
	data = data[data['RT'] < 7000]
	#print(data)
	print('Missing Values Removed: {}'.format(l - len(data)))
	#print(data[0:4])

	# Compute Signal Detection Theory (SDT) measures
	print('Level: Subject')
	r = data['Response']
	print('{} responses'.format(len(r)))
	hit = len(r[r == b'HIT'])
	miss = len(r[r == b'MISS'])
	fa = len(r[r == b'FA'])
	cr = len(r[r == b'CR'])
	total = hit + miss + fa + cr
	assert len(r) == total

	print('Total Valid Trials {}'.format(total))


	data_trials.append(total)

	hitrate = 100 * hit / total
	farate = 100 * fa / total

	data_hits.append(hitrate)
	data_fa.append(farate)

	print('Hits: {}\nMisses: {}\nFalse Alarms: {}\nCorrect Responses: {}'.format(hit,miss,fa,cr))

	# Rounding
	decimals = 2

	print('Hit{2}: {0}%\nFalse Alarm{2}: {1}%\n[of valid responses]'.format(round(hitrate, decimals),round(farate, decimals),'rate'))

	print('Level: Per Item in Subject')
	print('Items: {}'.format(items))

	#  Compute Cowan's K for all items
	 
	data = pd.DataFrame(data)

#	print(data)

	'''TODO Groupby Subject ID... Accumulate'''
	responses = data.groupby(['Items', 'Response']) # Yeah, I know it's Hungarian notation, because intuitively one would expect that pandas returned a dataframe
	responses = responses.size()




	# For each item get total and individual SDT values


	#agg_responses_per_items.loc[:,['CR']]
	#ai = np.nditer(a, flags=['multi_index'])
	# no clue what setting this flag or not does. preserves multi_index? #ainoflag = np.nditer(a)

	# Helper function
	fun = lambda x: np.asarray(x)

	#
	# Signal Detection Measures
	#
	#
	#
	#


	#totali = fun([sum(responses.loc[i,:]) for i in items])

	totali = hi = fai = cri = mi = []

	hi = fun([responses.loc[:, b'HIT']])
	fai = fun([responses.loc[:, b'FA']])
	cri = fun([responses.loc[:, b'CR']])
	mi = fun([responses.loc[:, b'MISS']])
	totali = [np.sum(hi), np.sum(fai), np.sum(cri), np.sum(mi)]
	totali = fun(totali)


	decimals = 2
	N = fun(items)


	responses = pd.DataFrame(responses)

	for i in items:
	#	i = str(i)
		totali = hi = fai = cri = mi = []

	#		i = eval('b\'' + str(item))

	try:
		hi = responses[0][i][b'HIT']
	except:
		hi = []
	try:
		fai = responses[0][i][b'FA']
	except:
		fai = []
	try:
		cri = responses[0][i][b'CR']
	except:
		cri = []
	try:
		mi = responses[0][i][b'MISS']
	except:
		mi = []

	hi = fun(hi)
	fai = fun(fai)
	totali = np.sum(hi) + np.sum(fai)# + np.sum(cri) +np.sum(mi)

#		print(responses.loc(responses['Response']))

	#		fai = fun([responses.loc['Response']['FA'] & responses.loc['Items' == i]])
		#cri = fun([responses.get['Response'] == 'CR' & responses.at['Items'] == i])
		#mi = fun([responses.get['Response'] == 'MISS' & responses.at['Items'] == i])
		#totali = [np.sum(hi), np.sum(fai), np.sum(cri), np.sum(mi)]
	totali = fun(totali)
	print('ih fi totali')
	print(hi)
	print(fai)
	print(totali)
	data_ck.append(np.round( ( (N * (hi - fai)) / totali ), decimals))

		

	# Debug Entry.
	#code.interact(local=locals())
	# Move this line around to debug the variable "workspace"



	# Rounding precision
#	decimals = 2 

	# False Alarm Rate (False Positive Rate)
	# NTS: Might be iteresting to look at the ratios here... assuming the limit corresponds to a quanta - i.e. a limited resource (limes being the quantum) - that can be partially distributed....
	#print('FAs: {}'.format(np.round(fai / totali * 100, decimals)))


	# The original
	# (H + CR - 1) * N

	# Vector of items
#	N = fun(items)

'''
	try:
		# Because we had fun, we can by means of symbolical math compute Cowan's K thus
		ck1 = np.round( ( (hi + cri - 1) / totali ), decimals)

		print('N, hi, fai, totali, decimals, cri')
		print(N, hi, fai, totali, decimals, cri)

		ck2 = np.round( ( N * (hi - fai) / totali ), decimals)
		k_zero = np.round( (ck1 + ck2) / 2 , decimals)
		global_total = sum(totali)

	except e:
		print('Exception ', e)
		fai = fai[0]
		totali = totali[0]
		cri = cri[0]
		ck2 = np.round( ( N * (hi - fai) / totali ), decimals)
		k_zero = np.round( (ck1 + ck2) / 2 , decimals)
		global_total = sum(totali)



		# Stratified samples of n trials weighted by evidence per item
		# Compute the item percentage of the total sample
		strata_weights = totali / global_total
		# Compute the mean percentage
		mean_weight = np.mean(strata_weights)
		# Center the weights around the mean percentage by subtracting the mean
		mean_centered_weights = strata_weights - mean_weight

		print(strata_weights)
		print(mean_centered_weights)
		ck_error = 1 - (ck1 + ck2) / 2

		new_weights = mean_centered_weights + 1
		print(new_weights)


		# Just checking if using numpy results in 32-bit mess or if 64bit precision is kept.
		# Great. Test passed.
		#print('{}{}'.format(type(new_weights[0]), type(strata_ck1[0])))

		# Finally, weigh Cowan's k+ and k-
		strata_ck1 = np.round(new_weights * ck1, decimals)
		strata_ck2 = np.round(new_weights * ck2, decimals)

		# For each item
		ones = fun(len(items) * [1])

		#lk_error = (ck2 - ck1) / 2

		# Print debug information
		print('Per Item (2, 3, 4, 5)')
		print('Total: {}\nHits: {}\nFalse Alarms: {}'.format(totali, hi, fai))
		print('Cowan''s k+: {}'.format(ck1))
		print('Cowan''s k-: {}'.format(ck2))
		print('Cowan''s k0: {}'.format(k_zero))
		print(CSL_LINES * '#')
		print('{}{}{}{}{}{}'.format(ck1, ck2, k_zero, ck_error, strata_ck1, strata_ck2))
		print(CSL_LINES * '#')
		print('k error: {}'.format(ck_error))
		print('k+ weighted by coefficents (stratified sampling): {}'.format(strata_ck1)) # NTS tratified might not be the most accurate term here
		print('k- weighted by coefficents (stratified sampling): {}'.format(strata_ck2))

		#### TO DO .... FIX THIS Read the pandas manual
	#	big_data = pd.DataFrame(data=[ck1, ck2, k_zero, ck_error, strata_ck1, strata_ck2])
	#	next_data = pd.DataFrame(data=[ck1, ck2, k_zero, ck_error, strata_ck1, strata_ck2])
	#	big_data = pd.DataFrame.append(big_data, next_data)

'''

print('All files processed.', 'Results',)
#big_data.T

[print(i) for i in data_trials]
[print(i) for i in data_hits]
[print(i) for i in data_fa]

print('Cowan\'sK')
[print(ck) for ck in data_ck]


# Output to uniquely neamd timestamped file goes here


Current file is aledie_py_2016-05-24 16-05-34.717000.txt...
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


TypeError: 'bool' object is not iterable